In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!unzip /content/drive/MyDrive/dataset.zip -d /content/

In [2]:
import os
import tensorflow as tf
from tensorflow.keras import models, layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
import cv2
import imghdr
from matplotlib import pyplot as plt
import numpy as np

In [3]:
gpus = tf.config.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [5]:
data = tf.keras.utils.image_dataset_from_directory('/content/dataset')

Found 11796 files belonging to 12 classes.


In [10]:
data = data.map(lambda x,y: (x/255,y))

In [13]:
train_size = int(len(data)*.7)
val_size = int(len(data)*.2)
test_size = int(len(data))-(train_size + val_size)

In [14]:
train = data.take(train_size)
val = data.skip(train_size).take(val_size)
test = data.skip(train_size+val_size).take(test_size)

In [15]:
model = Sequential()

In [16]:
model.add(Conv2D(16,(3,3),1,activation='relu', input_shape=(256, 256, 3)))
model.add(MaxPooling2D())

model.add(Conv2D(32,(3,3),1,activation='relu'))
model.add(MaxPooling2D())

model.add(Conv2D(32,(3,3),1,activation='relu'))
model.add(MaxPooling2D())

model.add(Conv2D(16,(3,3),1,activation='relu'))
model.add(MaxPooling2D())

model.add(Flatten())

model.add(Dense(256, activation='relu'))
model.add(Dense(12,activation='sigmoid'))

In [17]:
model.compile('adam',loss=tf.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])

In [ ]:
model.summary()

In [19]:
logdir = 'logs'

In [20]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [ ]:
hist = model.fit(train, epochs=20, validation_data = val,callbacks=[tensorboard_callback])

In [23]:
scores = model.evaluate(test)

38/38 [==============================] - 11s 36ms/step - loss: 0.3853 - accuracy: 0.9211


In [ ]:
# Use various charts, graphs or plots to show hist details

In [25]:
from tensorflow.keras.models import load_model
model_version = 1
model.save(f"/content/drive/MyDrive/model/{model_version}.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [26]:
y_pred = model.predict(test)

38/38 [==============================] - 7s 27ms/step


In [37]:
tf_lite_model = "image_classifier.tflite"

In [38]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)

In [39]:
tflite_model = converter.convert()

In [40]:
open(f"/content/drive/MyDrive/model/{tf_lite_model}", 'wb').write(tflite_model)

3305256